In [1]:
import sys
sys.path.append('/home/sensei/jupy-notebooks/PorterFarms/')
print("============================================")
print("/  AllTroughs is running.                  /")
print("============================================")
from datetime import datetime, timedelta
import pytz
import json
import copy
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import configparser

config = configparser.ConfigParser()
config.read("../../../analytics_secrets.ini")

_SLACK_TOKEN = config['slack']['token']
_CHIRPSTACK_USER = config['chirpstack']['user']
_CHIRPSTACK_PASS = config['chirpstack']['password']

_DB_HOST = config['kanjidb']['dbhost']
_DB_PORT = config['kanjidb']['dbport']
_DB_NAME = config['kanjidb']['dbname']
_DB_USER = config['kanjidb']['dbuser']
_DB_PASS  = config['kanjidb']['dbpass']

_LOG_DEBUG = 0
_LOG_INFO  = 1
_LOG_ERROR = 2
_LOG_LEVEL = int(config['DEFAULT']['loglevel'])
def logger(level, message):
    if level >= _LOG_LEVEL:
      print(message)

logger(_LOG_DEBUG, _SLACK_TOKEN)
logger(_LOG_DEBUG, "{} {} {} {} {}".format(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS))

import kanjiticketing as kt

conn = kt.getKanjiDbConnection(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS)
if conn is not None:
  print("Welcome to Jupyter Notebook.  You are connected to the Kanji database!")
else:
  print("You are not connected to the database.")

/  AllTroughs is running.                  /
Python version
3.7.2 (default, Dec 29 2018, 06:19:36) 
[GCC 7.3.0]
Version info.
sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)
Welcome to Jupyter Notebook.  You are connected to the Kanji database!


In [2]:
_CATTLE_TROUGH_MONITOR = 10004
_INTERVAL_MINUTES = 20
_AGE_THRESHOLD_SECONDS = 900

#Ticket Type
LOW_WATER_LEVEL = 10001

now = datetime.now(pytz.utc)  #tz Aware
starttime = now - timedelta(hours=0, minutes=_INTERVAL_MINUTES)
logger(_LOG_DEBUG, "Current time is {}".format(now))
logger(_LOG_DEBUG, "Query timestamp will start at {}".format(starttime))

_THRESHOLD_WMA = 0.78

logger(_LOG_INFO,"Threshold wma={:3.2f}".format(_THRESHOLD_WMA))

nodequery = "SELECT * FROM kanji_node WHERE application_id={};".format(_CATTLE_TROUGH_MONITOR)
df = pd.read_sql(nodequery, conn)

logger(_LOG_DEBUG, "number of trough nodes {}".format(len(df.index)))
for ind in df.index:
  node_id = df['idnode'][ind]
  eventquery = "SELECT * FROM kanji_eventlog WHERE node_id={} AND sensortype_id=39 AND timestamp > '{}' ORDER BY timestamp desc;".format(node_id, starttime)
  logger(_LOG_DEBUG, eventquery)
  df2 = pd.read_sql(eventquery, conn)
  samplesize = len(df2.index)
  logger(_LOG_DEBUG, "samplesize {}".format(samplesize))
  wma = 0.0
  divisor = 0  
  for ind2 in df2.index:
    #logger(_LOG_ERROR, df2['timestamp'][ind2])
    data = json.loads(df2['eventdata'][ind2].replace("'","\"")) 
    if ind2==0:       
      logger(_LOG_DEBUG, data)
      latestsensorstatus = float(data['currentstatus'])  
      logger(_LOG_DEBUG,"latestsensorstatus ={}".format(latestsensorstatus))
    sensorstatus = float(data['currentstatus'])
    wma += sensorstatus * (samplesize - ind2)
    divisor += (samplesize - ind2)
    logger(_LOG_DEBUG, "status {} wma {}".format(sensorstatus, wma))
  wma = wma/divisor
  logger(_LOG_INFO, "wma={}".format(wma))  
  if wma>=_THRESHOLD_WMA:
    logger(_LOG_INFO, "WMA at or above crtical level {}".format(wma))
  elif latestsensorstatus==0.0:
    #trigger an alert ONLY if the sensor is DRY'
    nodename = df['name'][ind]
    location_id = df['location_id'][ind]
    locationquery = "SELECT location.idlocation, location.description, location.imageurl, location.slackchannel, \
                     customer.slacktoken \
                     FROM kanji_location location \
                     JOIN kanji_customer customer ON location.customer_id=customer.idcustomer \
                     WHERE idlocation={}".format(location_id)
    df3 = pd.read_sql(locationquery, conn)
    locationid = df3["idlocation"][0]
    locationimageurl = df3["imageurl"][0]
    logger(_LOG_DEBUG, locationimageurl)
    locationdescription = df3["description"][0]
    _SLACK_TOKEN = df3["slacktoken"][0]
    logger(_LOG_DEBUG, _SLACK_TOKEN)
    _SLACK_CHANNEL = df3["slackchannel"][0]
    
    logger(_LOG_DEBUG, "locationdata")
    logger(_LOG_DEBUG, "locationQuery={}".format(locationquery))
    logger(_LOG_INFO, "below threshold")
    description = "critical water level. ({:3.1f})".format(wma)
    mentions = " @Charlie, @Jared"
    #generate and Slack a new ticket ONLY if there is not a currently open ticket for this issue
    openTicket = kt.ticketExists(conn, node_id, LOW_WATER_LEVEL, [kt._OPEN_STATUS, kt._WORKING_STATUS])
    if openTicket is None:
      ticketid = kt.openticket(conn, node_id, locationid, description, 2, 3, LOW_WATER_LEVEL, _SLACK_CHANNEL)
      ts = kt.slackticket(nodename, locationdescription, description, mentions, 2, 3, locationimageurl, _SLACK_TOKEN, _SLACK_CHANNEL, ticketid, 0)
      kt.updateTicket(conn, ticketid, ts)  
      logger(_LOG_INFO, "New ticket {} created for this issue.".format(ticketid))
    else:
      logger(_LOG_DEBUG, "There is an existing ticket {} for this issue. {}".format(openTicket['idticket'][0], openTicket['opentimestamp'][0]))
      age = now - openTicket['lastupdatetimestamp'][0]
      ageseconds =  age.days*24*60*60 + age.seconds
      #Calculate age of ticket and reissue if it is stale
      logger(_LOG_DEBUG, "ageseconds ={}".format(ageseconds))
      if ageseconds >= _AGE_THRESHOLD_SECONDS:
        description = "REISSUE ALERT critical water level. ({:3.1f})".format(wma)
        #ticketid = openticket(node_id, locationid, description, 2, 3, LOW_WATER_LEVEL)
        #kt.updateTicket(conn, openTicket['idticket'][0], openTicket['ticketts'][0])
        #kt.slackticket(nodename, locationdescription, description, mentions, 2, 3, locationimageurl, _SLACK_TOKEN, _SLACK_CHANNEL, openTicket['idticket'][0], openTicket['ticketts'][0])
        #logger(_LOG_INFO, "Alert reissued for ticket {}.".format(openTicket['idticket'][0]))
  else:
    logger(_LOG_INFO, "WMA below threshold, but sensor is currently WET")
    

Threshold wma=0.78
wma=0.0
below threshold
Checking tickets for 20002 10001 [10000, 10001]
Status clause (status_id=10000 OR status_id=10001)
SELECT * FROM kanji_ticket WHERE node_id=20002 AND type_id=10001 AND (status_id=10000 OR status_id=10001) ORDER BY opentimestamp DESC
No ticket found.
INSERT INTO kanji_ticket (opentimestamp, lastupdatetimestamp, node_id, location_id, description,                  impact_id, urgency_id, type_id, status_id, ticketchannel, ackuser_id)                  VALUES ('2020-05-20 11:48:41.834028', '2020-05-20 11:48:41.834036', '20002', '10013', 'critical water level. (0.0)', 2, 3, 10001, 10000, 'trough', 10006) 
307
https://api.slack.com/img/blocks/bkb_template_images/notifications.png
Ok posting message to Slack channel
Message ts = 1589989722.021000
UPDATE kanji_ticket SET lastupdatetimestamp = NOW(), ticketts='1589989722.021000' WHERE idticket=307
New ticket 307 created for this issue.
